#Lab.07 / IBM3202 – Molecular Dynamics on GROMACS

###Theoretical aspects

As we discussed in class, the core mechanism of **molecular dynamics (MD)** is **numerically solving Newton’s equation of motion** by deriving the **potential energy** (i.e. the energy of bonded and non-bonded interactions) for each atom during propagation of their changes in position upon time. These changes in position are the consequence of both **atom-atom interactions** and **thermal motions** (kinetic energy).

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/MD_01.png'/>
<figcaption>FIGURE 1. The performance (ns/day) of molecular dynamics simulations depend on (A) the number of atoms of the simulated system and (B) the use of CPU alone or combined with GPU nodes for performing the simulations. <br> Kutzner C et al (2014) <i> IOS Press, 722-730</i>; Kutzner C et al (2019) <i> J Comput Chem 40(27), 2418-2431</i></figcaption></center>
</figure>

As most of the potential energy is a pairwise calculation, with **most of these calculations corresponding to non-bonded interactions** (and requiring switching functions and distance cut-off strategies to reduce their computational costs), the **time required for solving these equations exponentially increases with the number of particles in the system**. In previous years, these computational needs were typically alleviated with costly hardware investments in CPU computing power. But during the last years, several MD simulation packages have been developed to compute most (if not all of) **non-bonded interactions** on consumer-grade GPUs, off-loading the CPU and generating significant speedups in affordable computers.

Luckily for us, the advent of **cloud computing** and the emergence of **Google Colab** heavily relies on the use of GPUs. Thus, these cloud services can be efficiently use to perform **10 to 100 ns-long MD simulations**.



##Experimental Overview

Inspired by the COVID-19 pandemic, in this laboratory session we will perfom an MD simulation of the **papain-like protease of SARS-CoV-2**, a current drug-design target to combat this virus.

For our laboratory session, we will compile and install **GROMACS**, an MD simulation package that we will use to set-up and perform our simulations. We will visualize our protein structure using **py3Dmol**, while the simulation trajectories from our MD production runs will be visualized in a web version of **NGLview**. We will also analyze some parameters regarding the proper equilibration of our simulation system, but a deeper analysis of protein motions will take case in the next tutorial.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/MD_02.jpg' />
<figcaption>FIGURE 2. General steps for performing MD simulations. Starting from a experimental or modelled protein structure, a simulation system is prepared by setting up a simulation box, solvating the protein with water and neutralizing the total charge of the system with counterions. Then, the simulations are performed, visualized and analyzed in terms of their protein motions and energetic features<br>Morgnanesi D et al (2015) <i>Antiviral Res 123, 204-215</i></figcaption></center>
</figure>

#Part 0. Downloading and Installing the required software

Before we start, you must:
1. Remember to **start the hosted runtime** in Google Colab.
2. **VERY IMPORTANT‼️** Go to *Runtime* -> *Change Runtime Type* and select GPU.

Then, we must install three pieces of software to perform this tutorial. Namely:
- **biopython** for manipulation of the PDB files
- **py3Dmol** for visualization of the protein structure.
- **GROMACS** for preparing our MD system and performing our MD simulations.


For visualizing our MD trajectories, we will employ a web version of **NGLview**. This is due to the inability of Google Colab to handle a required python package for loading NGLview directly onto Google Colab. Hopefully this will change in the near future.
The analysis of our trajectories will be mostly performed on the next laboratory session!

1. We will first start by installing **py3Dmol** as follows:

In [ ]:
#Installing py3Dmol using pip
!pip install py3Dmol

In [ ]:
#Importing py3Dmol for safety
import py3Dmol

3. Next, we will install **biopython**

In [ ]:
#Installing biopython using pip
!pip install biopython

3. Finally, we will install **GROMACS** on Google Colab using **conda**

In [ ]:
#Install conda using the new conda-colab library
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

#Install GROMACS 2024.3 from the conda-forge repository
!conda install -c conda-forge gromacs=2024.3=nompi_cuda_h5cb645a_1 ncurses -y

Once these software installation processes are completed, we are ready to perform our experiments

# Part I – Setting up the MD simulation system

GROningen Machine for Chemical Simulations (**GROMACS**) is an open-source, free software developed by the University of Groningen with consistent and continuous support. It also has been optimized for calculations by maximizing the usage of all available computational resources (GPU and CPU). Once installed, you can access all the modules it has for setting up, running and analyzing MD simulations by just inputting:

**`gmx [module]`**

The list of modules can be displayed when **`[module]=help`**. Try it out here!

In [ ]:
%%bash
gmx -h

The initial steps of setting up a system for MD simulations are:

1. **Cleaning up** the input atomic coordinates
2. **Parameterizing** the atoms building up our system
3. **Solvating** our protein (and adding a lipid membrane in the case of membrane proteins)
4. **Adding** counterions to neutralize the global charge of the system

### Part I.1 – Cleaning up the input atomic coordinates

For this step, we first need an initial set of atomic coordinates, usually coming from a protein structure downloaded from the Protein Data Bank. Once download, these PDB files must be cleaned from water molecules (in the case of crystal structures) or a single model must be selected from the many solutions for a single set of chemical shifts (in the case of NMR structures).

1. We will first start by making a folder for preparing our system and running our simulations, as we have done in the past (remember our Molecular Docking tutorial?)

In [ ]:
#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path

#Then, we define the path of the folder we want to create.
#Notice that the HOME folder for a hosted runtime in colab is /content/
mdpath = Path("/content/md01/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(mdpath):
  print("path already exists")
if not os.path.exists(mdpath):
  os.mkdir(mdpath)
  print("path was succesfully created")

2. Then, we will change directory to this newly created folder and download the recently X-ray solved structure of the papain-like protease (PLpro) of SARS-CoV-2 (PDB 6WZU).

In [ ]:
#First, we will change to the new folder. We will use python now :)
os.chdir(mdpath)

In [ ]:
#Importing your PDB file using biopython
import os
from Bio.PDB import *
pdbid = ['6wzu']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

3. This structure has a few residues in two different conformations (remember when we discussed **occupancy** in classes and during our second tutorial?). We can filter out one of them,  

In [ ]:
#Here we set up a parser for our PDB
parser = PDBParser()
io=PDBIO()
structure = parser.get_structure('X', '6wzu.pdb')
#And here we set the residue conformation we want to keep
keepAltID = "A"

class KeepOneConfOnly(Select):  # Inherit methods from Select class
    def accept_atom(self, atom):
        if (not atom.is_disordered()) or atom.get_altloc() == keepAltID:
            atom.set_altloc(" ")  # Eliminate alt location ID before output.
            return True
        else:  # Alt location was not one to be output.
            return False
        # end of accept_atom()

#This will keep only conformation for each residue
io.set_structure(structure)
io.save("6wzu_ready.pdb", select=KeepOneConfOnly())
print("Your PDB was processed. Alternative side chain conformations removed!")

4. Commonly, crystallographic waters and other solvent molecules are stripped from the experimentally solved structure before preparing the MD system. This is also useful for removing hydrogens, which are often found in NMR structures, as they are a nightmare to pass through the atomic conventions of a given force field. We can do this using `Dice` from **biopython** as show in the code cell bellow.

❗️**NOTE:** This procedure is not universally appropriate. Some water molecules could be essential for the function of a protein and thus maintained in the simulation. Also, `Dice` removes ligands, which in some cases need to be maintained depending on the goal of your simulations and the ability to parameterize your ligands. Sometimes, it will be more useful to use `grep`. For example:

  `!grep -v "HOH" 6WZU_ready.pdb > 6WZU_clean.pdb`
  
  This grep command would remove only water molecules from the PDB file.

In [ ]:
#Here we set up a parser for our PDB
parser = PDBParser()
io=PDBIO()
structure = parser.get_structure('X', '6wzu_ready.pdb')
#And here we remove hydrogens, waters and ligands using Dice
io.set_structure(structure)
sel = Dice.ChainSelector('A', 1, 5000)
io.save("6wzu_clean.pdb", sel)
print("Your PDB was processed. Only the protein heavy atoms have been kept!")

5. Let's load the protein that we are working with using py3dmol

In [ ]:
#First we assign the py3Dmol.view as view
import py3Dmol
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files
view.addModel(open('6wzu_clean.pdb', 'r').read(),'pdb')
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style and color
view.setStyle({'chain':'A'},{'cartoon': {'color':'spectrum'}})
#Centering view on all visible atoms
view.zoomTo()
#And we finally visualize the structures using the command below
view.show()

### Part I.2 – Parameterizing the atoms building up our system

Now, we will work with GROMACS to parameterize our protein, generating:

*   A **.gro** or **.pdb** coordinate file that contains all the atom types as defined by a given force field (including hydrogens).
*   A **.top** topology file containing the parameters for bonds, angles, dihedrals and non-bonded interactions defined by a given force field (potential energy function) to employ in our simulations.

1. We will parameterize our protein using the **AMBER99SB-ILDN force field** on GROMACS and obtain these files using `gmx` as shown in the code cell below. This force field is extensively used in MD simulations and has parameters that well-represent the dynamics and flexibility of folded proteins. Notice that the dynamics of highly motile proteins or intrinsically disordered regions is not the main dataset for which this force field was parameterized, and other options may better suit such goals.

In [ ]:
%%bash
#Using pdb2gmx to parameterize our PDB with the AMBER forcefield and SPC/E water
gmx pdb2gmx -f 6wzu_clean.pdb -o 6wzu_processed.pdb -water spce -ignh -ff amber99sb-ildn

When using this command, the `-f` option corresponds to the input file, the `-ff` option allows to define the force field to be used and the `-o` option provides the output name for the .coordinate file. The topology file receives a default `topol.top` name, although this can be altered with the `-p` option.

We are also indicating the type of water molecule that we will be using in our simulations through the `-water` option. Yes, there are many water models, but their features are outside the scope of our tutorial. Our selected water model is SPC/E (simple point charge - extended), which models water molecules as a 3-point molecule, each with its own charge. TIP3P is a more common water model, similar to SPC/E but with slightly different parameters. The accuracy of the model used changes depending on the calculations that you will perform, as their dynamic and energetics will differ.

❗️**NOTE:** When working with NMR files, it is useful to incorporate the `-ignh` option, that allows to ignore hydrogens contained in the PDB file. During parameterization, hydrogens are always added back by the force field, as it contains the typical distance parameters for all atoms based on experimental/quantum mechanics analysis.

### Part I.3 – Solvating our protein

We will now define a periodic box for our simulation system, in which our protein will be centered ,and then fill this box with water molecules, thus solvating our protein.

Given the use of **periodic boundary conditions** and the computational costs of the evaluation of non-bonded interactions, it is imperative to properly define the distance of the periodic box such that it is large enough to avoid interactions between our molecule and its periodic images, while at the same time minimizing the number of water molecules (in essence, the number of non-bonded interactions to evaluate during the simulation).

It is often recommended for globular proteins to have a **padding distance** between the furthest protein atom from the center of the periodic box and its edges of 1.0-1.5 nm. If your system has one dimension much larger than the others (i.e. fibrillar proteins) you may wish to draw this periodic box more carefully.

1. First, we will generate a periodic box using `editconf` as follows:

In [ ]:
%%bash
#Using editconf to create a cubic box with 1.0 nm padding for our solvated system
gmx editconf -f 6wzu_processed.pdb -o 6wzu_newbox.pdb -c -d 1.0 -bt cubic

The options provided to `editconf` enable to center (`-c`) the atom coordinates of our protein (`-f`) in a cubic box (`-bt cubic`) with a padding distance of 1.0 nm from the box edge (`-d 1.0`).

Given periodic boundary conditions, this means that the system will be at the closest 2.0 nm from itself, in which most of the non-bonded terms are out of range between periodic images due to the use of distance cut-offs. Thus, the only molecules that will be interacting between the edges of the periodic box are water molecules and ions. These new coordinates and simulation box are saved in a new coordinate file (`-o`).

2. Once the periodic box is defined, we proceed to fill this box with water molecules using `solvate`. To do this, we execute:

In [ ]:
%%bash
#Using solvate to fill up our box with water molecules
gmx solvate -cp 6wzu_newbox.pdb -o 6wzu_solv.pdb -p topol.top

Please note that, given the addition of water molecules to our simulation system, we are generating **a new topology file** (`-p` option) and **a new coordinate file with added water molecules** (`-o` option).

Whenever we write a new topology file (or any file with the same name), GROMACS backups the previous file by adding a `#` symbol (i.e. `#topol.top.1`) and then generates the new .top file. This way you do not have to worry about backing up the files.

**QUESTION❓:** From reading the output of the previous command, how many water molecules were added?

3. Let's look at the solvated system using py3Dmol! How are the water molecules solvating the protein? Are the molecules homogeneously distributed?

In [ ]:
#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files
view.addModel(open('6wzu_solv.pdb', 'r').read(),'pdb')
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style and color
view.setStyle({'cartoon': {'color':'green'}})
#Here we add a style for showing the oxygen from water molecules
view.addStyle({'atom':'OW'},{'sphere':{'radius':'0.2'}})
#Centering the view on all visible atoms
view.zoomTo()
#And we finally visualize the structures using the command below
view.show()

## Part I.4 – Adding counterions to neutralize the global charge of the system

Now we have a solvated box, but our system has a **non-zero charge**. This is an issue, as each periodic box will have a formal charge, which would cause electrostatic potentials to become unrealistically high among periodic images. Thus, we need to neutralize the charges of our simulation system.

1. Let's easily determine the absolute charge in our system by using `grep`

In [ ]:
!grep "qtot" topol.top | tail -n 1

**QUESTION❓:** What is the total charge of our system?

To neutralize the simulation system, we will replace water molecules with the counterions required to get the absolute charge of the system to **zero**. While this might be sufficient for globular proteins in water, in some cases you might find more useful to reach a particular salt concentration or generate charge differences, for example in the case of simulations of ion channels in a lipid bilayer.

2. The addition of counterions is achieved by building a **portable binary run input file for GROMACS**, or **.tpr file**. This file contains the information from the coordinates, topology/parameters and an **MD instruction file** (**.mdp**). This instruction file contains all the parameters that will be used for running different calculations, and you can input flags that will be read by those specific programs.

  We will download and use a predefined **ions.mdp** file (**please take a look at it!**) onto our simulation folder and generate the .tpr run input file by executing the following command with the `grompp` module:

In [ ]:
%%bash
wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/ions.mdp

#Using grompp and an MD instruction file to add counterions to our system
gmx grompp -f ions.mdp -c 6wzu_solv.pdb -p topol.top -o ions.tpr

Please note that, in the case of `grompp`, the `-f` option is now used for the .mdp file, while `-c` is used for the coordinates.

3. Once this is done, run the `genion` module for randomly replacing water molecules in your simulation system with an appropriate number and type of counterions.

❗️**NOTE:** Usually `gmx` would **interactively request us** to choose a group for this replacement (in our case, group 13 "SOL", corresponding to water molecules). Here, we are solving the impediment of doing this on Google Colab by **generating a text file with these options**, which then we pass onto `gmx` to be read

In [ ]:
%%bash
#This is a trick to provide interactive options to gmx
echo "SOL" > options
echo " " >> options

#Using genion and the tpr to add counterions to our solvated system
gmx genion -s ions.tpr -o 6wzu_solv_ions.pdb -p topol.top -pname NA -nname CL -neutral < options

**QUESTION❓:** What type of monovalent ion was added? How many ions were added?

We have finalized preparing the simulation system with our protein in the center of a cubic box with 1.0 nm of padding, solvated in water molecules and neutralized with counterions. You can choose to visualize it using py3Dmol if you want!

#Part II – Minimizing and Equilibrating the MD system

Now, we are ready to perform the minimization of our system to eliminate the high energy and forces due to bad initial coordinates, and its equilibration at constant pressure and temperature (NPT ensemble).

1. We will start by downloading an MD instruction file that contains all of the parameters required for the minimization of our system. We will also print its contents for inspection.

In [ ]:
%%bash
wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/em.mdp

In [ ]:
%%bash
#Check the content of the MDP file
paste em.mdp

As you can see, the minimization process involves the use of a **steepest descent** (`steep`) integrator. This integrator, alongside conjugate gradient (`cg`) and a quasi-Newtonian method (`l-bfgs`), are **minimization algorithms** that instead of solving the positions for changes in the gradient (Newton’s equation of motion), look for changes in position that would **minimize the potential energy**.

These minimization (sometimes referred to relaxation) protocols make sure that the **starting point from our simulations has a ΔU favorable enough** for Verlet and leapfrog algorithms (which enable the **propagation of changes in atom positions for each time step**) to be numerically integrable.

In simpler words, this step is critical as starting from a high potential energy can generate from highly divergent simulations, in the mildest of cases, to energetic instabilities that would make our system impossible to numerically integrate (typically known as a **“system that explodes”**).


2. We will pre-process our files with `grompp` and then run our MD with `mdrun`. Check the resemblance (and also differences) of the use of `grompp` for adding neutralizing counterions and the system minimization.

In [ ]:
%%bash
#Using grompp to prepare our minimization MD
gmx grompp -f em.mdp -c 6wzu_solv_ions.pdb -p topol.top -o em.tpr

#Run our minimization
gmx mdrun -v -deffnm em -nb gpu

For `mdrun`, we are using the verbose option (`-v`) to output the changes in forces between atoms during the system minimization. As the system is minimized, these forces should be reduced when compared to the initial system.

The `-deffnm` option simply states that all MD-generated files will have the same standard name (in this case, *em*) as the .tpr file generated with `grompp`

**📚HOMEWORK:** Look up which files have been generated after `mdrun`. What is inside each generated file? If you do not know, **Google it!** This step is **obligatory**.

😱 **EMERGENCY BACKUP!** Ran out of time and could not minimize your system? Use the following code cell to download a readily minimized system:

In [ ]:
#Use only in case of emergency
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/emergency_backup/lab07/6wzu_em.tar.gz
!tar xzf 6wzu_em.tar.gz

3. Once our minimization is finished, we can check how the potential energy of the system changes over each minimization step. For this, we will use the `energy` module to extract the potential energy, and then we will plot it using **matplotlib**:

In [ ]:
%%bash
#This is a trick to provide interactive options to gmx
echo "Potential" > options
echo " " >> options

#Using energy to extract the potential energy of the system
gmx energy -f em.edr -o em_potential.xvg -xvg none < options

In [ ]:
#Plotting the potential energy of the system
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
import numpy as np

#Reading the text file containing this information
data = np.loadtxt('em_potential.xvg')

plt.title('Potential Energy during Minimization')
plt.xlabel('Energy Minimization Step')
plt.ylabel(r'E$_P$ [kJ•mol$^{-1}]$')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

Next, we will start our **equilibration**. This step is required given that we optimized the positions of the atoms according to our force field, but at no point we maximized the interactions between solvent and solute. We also have not established the **kinetic effect of temperature on atom motions** or the **pressure of our system**, thus making it unrelatable to experimental conditions.

Thus, we will **equilibrate the energy and density of our system at constant temperature and pressure** before the MD production runs.

4. The first equilibration step is the **temperature**. Thus, we will start a MD simulation to equilibrate the system at a target temprature (in our case, 300K) using a thermal bath. The initial velocities for the atoms of our system at our target temperatureare obtained through a Maxwell distribution.

  We will first download an appropriate MD instruction file and then run our simulation as we did before for the minimization. Also note that we are now using **GPU resources** for the non-bonded interactions.

In [ ]:
%%bash
wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/nvt.mdp

#Using grompp to prepare our NVT equilibration MD
gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

In [ ]:
%%time
%%bash
#Run our NVT equilibration MD
gmx mdrun -deffnm nvt -nb gpu

😱 **EMERGENCY BACKUP!** Ran out of time and could not perform the NVT equilibration of your system? Use the following code cell to download a temperature-equilibrated system:

In [ ]:
#Use only in case of emergency
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/emergency_backup/lab07/6wzu_nvt.tar.gz
!tar xzf 6wzu_nvt.tar.gz

If you take a look at the MD instruction file for this step, you will notice that we are now using a **leapfrog algorithm** (`md`) for propagation of the protein motions upon time.

 Among several parameters, we can read that the **hydrogen atoms are constrained**. This is because those nuclei have little mass and are commonly inside the Lennard-Jones radius of a molecule, lack any type of dihedral motions, and the distance of the bond is highly conserved. This is why constraint algorithms are used to simplify their movement equations, while keeping the effect they have on solvation and interactions. Given that the faster internal motions correspond to the bond vibrations between light atoms (such as hydrogen), their constraining enables the use of **higher timesteps** (in our case, we will use **2 fs**).

Finally, in this equilibration step a **position restraining algorithm** is used to keep the heavy atoms of our protein in place. This is because we know our protein structure is mostly ideal (because it is modeled from experimental data), but our solvent molecules were just drawn around it. So, in this step we will keep our protein mostly in place and let the solvent rearrange around our protein as we would expect it to do at 300 K. This information is in the **posre.itp** (a position restrain file) that was generated at the beginning of the use of GROMACS. This restrain is performed via the `-r em.gro` option.


5. What we just did corresponds to a simulation setup in which the number of atoms, the volume and the temperature are kept constant: **NVT ensemble**. Thus, the temperature of the system should oscillate around the desired temperature (in our case, 300 K). Let's check if this is true!

In [ ]:
%%bash
#This is a trick to provide interactive options to gmx
echo "Temperature" > options
echo " " >> options

#Using energy to extract the temperature of the system during the NVT equil MD
gmx energy -f nvt.edr -o nvt_temp.xvg -xvg none < options

In [ ]:
#Plotting the temperature of the system
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
import numpy as np

#Reading the text file containing this information
data = np.loadtxt('nvt_temp.xvg')

plt.title('Temperature during 0.1 ns Equilibration (NVT)')
plt.xlabel('Time (ps)')
plt.ylabel('Temperature [K]')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

6. The final equilibration step will allow us to implement **pressure regulation** to our system, which will maintain the density and pressure of our solvent constant so that it matches what we would expect for the SPC/E water model. Thus, in this case we will be using an ensemble in which the number of atoms, the pressure and the temperature of the system remain constant: **the NPT ensemble**.

  We will perform the same steps we did before: download an MD instruction file, and then use `grompp` and `mdrun` to run the MD simulation. One big difference between this simulation phase and the previous ones is that we will employ a **checkpoint state file** (nvt.cpt), which allows for a numerical continuation of our simulations. This file constitutes a checkpoint of the previous NVT simulation, containing the potential energy of the system and the velocities.

In [ ]:
%%bash
wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/npt.mdp

#Using grompp to prepare our NPT equilibration MD
gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

In [ ]:
%%time
%%bash
#Run our NPT equilibration MD
gmx mdrun -deffnm npt -nb gpu

**⚠️CAUTION:** We used a Parrinello-Rahman barostat instead of Berendsen for the NPT equilibration. This should not be an issue if the system is adequately equilibrated, otherwise the combination of restraints and Parrinello-Rahman would be inappropriate as it could lead to instabilities.

😱 **EMERGENCY BACKUP!** Ran out of time and could not perform the NPT equilibration of your system? Use the following code cell to download a pressure-equilibrated system:

In [ ]:
#Use only in case of emergency
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/emergency_backup/lab07/6wzu_npt.tar.gz
!tar xzf 6wzu_npt.tar.gz

7. Given that we are using an NPT ensemble to maintain our simulation at constant pressure and density, we should check if this is achieved.

In [ ]:
%%bash
#This is a trick to provide interactive options to gmx
echo "Pressure" > options
echo "Density" >> options
echo " "

#Using energy to extract the pressure and density of the system during the NPT equil MD
gmx energy -f npt.edr -o npt_press_dens.xvg -xvg none < options

In [ ]:
#Plotting the density of the system
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
import numpy as np

#Reading the text file containing this information
data = np.loadtxt('npt_press_dens.xvg')

plt.title('Pressure during 0.1 ns Equilibration (NPT)')
plt.xlabel('Time (ps)')
plt.ylabel('Pressure [bar]')
plt.ylim(-250,250)

#Smoothing using Savitzky-Golay
from scipy.signal import savgol_filter
yhat = savgol_filter(data[:,1], 21, 5)

#Plot raw data and spline interpolation
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.plot(data[:,0], yhat, linestyle='solid', linewidth='2', color='blue')
plt.show()

In [ ]:
#Plotting the pressure of the system
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
import numpy as np

#Reading the text file containing this information
data = np.loadtxt('npt_press_dens.xvg')

plt.title('Pressure during 0.1 ns Equilibration (NPT)')
plt.xlabel('Time (ps)')
plt.ylabel('Density [kg•m$^{-3}$]')
plt.ylim(1000,1020)
plt.plot(data[:,0], data[:,2], linestyle='solid', linewidth='2', color='red')
plt.show()

**QUESTION❓:** What is the average value of the density along the trajectory? Sounds familiar?

Once this section is finished, we can move onto production MD simulations.

#Part III – Obtain a production MD run and analyze the results

Now we are ready for running our MD simulations. If we are using a typical NPT ensemble, we can just continue our simulation similar to our second equilibration, but without having any position restraints (without the `-r` option).

1. If you have paid attention to our previous MD runs, you will fully understand what is annotated in the following code cells. Due to time constraints and the size of the system, we are only generating **0.1 ns of production runs**, whilst you will find that simulations in current articles often correspond to hundreds of ns.

In [ ]:
%%bash
wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/md.mdp

#Using grompp to prepare our production MD
gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_1.tpr

In [ ]:
%%bash
#Check the content of the production MD file
paste md.mdp

In [ ]:
%%time
%%bash
#Run our production MD
gmx mdrun -deffnm md_1 -nb gpu

2. To finalize, we will visualize our simulation. For this, we will use the `trjconv` module to extract only the protein from our system and convert our trajectory into a PDB file.

  Given that our protein may have drifted from the center of the box and reached its edges, we will also take the opportunity to recenter the periodic box on our protein such that its atoms do not stretch through the edges of the periodic boundary conditions using the `-pbc mol` and `-center` options.

In [ ]:
%%bash
#This is a trick to provide interactive options to gmx
echo "Protein" > options
echo " " >> options
echo "Protein" >> options
echo " "

#Using trjconv to extract only the protein atoms from the simulation trajectory
#and also recenter the protein if its atoms crossed the periodic boundaries
gmx trjconv -s md_1.tpr -f md_1.xtc -o md_1_protPBC.pdb -pbc mol -center < options

3. Now, you can download this new PDB file and load it onto [**NGLviewer**](http://nglviewer.org/ngl/) as a **trajectory** PDB file to visualize the protein motions explored within this short simulation time.

😱 **EMERGENCY BACKUP!** If you did not obtain the production MD trajectory on time, you can download <a href="https://github.com/pb3lab/ibm3202/blob/master/files/emergency_backup/lab07/md_1_protPBC.pdb" target="_blank"/>this 0.2 ns trajectory</a>.

4. Now we will analyze two global structural parameters that are often reported in many articles utilizing MD to characterize protein motions: the root mean squared distance (RMSD) and the per-residue root mean square fluctuations (RMSF). Both parameters are calculated for all frames in the trajectory against a reference structure, which we will extract from the final frame of the equilibration at constant pressure (NPT).

In [ ]:
%%bash
#Generating a reference structure for the RMSD
echo "Protein" > options
echo " " >> options
echo "Protein" >> options
echo " "
gmx trjconv -s npt.tpr -f npt.gro -o npt_protPBC.pdb -pbc mol -center < options
#Commands for RMSD
echo "CA" > options
echo " " >> options
echo "CA" >> options
echo " " >> options
#RMSD calculation
gmx rms -s npt_protPBC.pdb -f md_1_protPBC.pdb -xvg none < options
#RMSF calculation
gmx rmsf -s npt_protPBC.pdb -f md_1_protPBC.pdb -xvg none -res < options


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('rmsd.xvg')

plt.title('C-alpha RMSD')
plt.xlabel('Time (ps)')
plt.ylabel('RMSD [nm]')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('rmsf.xvg')

plt.title('C-alpha per-residue RMSF')
plt.xlabel('residue number')
plt.ylabel('RMSF [nm]')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

**And this is the end of the seventh tutorial!** On the next tutorial, we will learn how to perform some analysis of our MD production runs.

If you want to download your results, you can perform the following commands:

In [ ]:
os.chdir("/content/")
!zip -r md.zip $mdpath
from google.colab import files
files.download("/content/md.zip")
os.chdir(mdpath)

**📚HOMEWORK:** Once you got here, you can perform several exercises!

1.	Try to perform the equilibration without minimization of the simulation system (i.e. skip the use of em.mdp and go straight onto equilibration).
2.	Change the timestep from 2 fs to 4 fs or 5 fs, and check what happens with the simulation system in terms of stability. If you get some warnings and no .tpr file is compiled, you can add the `-maxwarn X` option, where `X` is the number of warnings from GROMACS that you will ignore.

With these examples, it will be possible to better understand why defining the proper timestep and adequately minimize and equilibrate the system are important steps in all MD simulations.

3. Run your simulation for a longer time (for example, 1 ns; ~ 1 h on Google Colab).